In [0]:
%run "./Configuration"

In [0]:
# Define the metadata widget:
dbutils.widgets.text("entityName", "")
dbutils.widgets.text("entityColumns", "")
dbutils.widgets.text("sourceSystem", "")

# Save parameters to variables:
entityName =       dbutils.widgets.get('entityName')
sourceSystemName = dbutils.widgets.get('sourceSystem')
columns =          json.loads(dbutils.widgets.get('entityColumns'))
raw_path =         entity_bronze(entityName)

In [0]:
df = spark.read.format("parquet").load(BRONZE + raw_path)

In [0]:
df.createOrReplaceTempView(f'tv_{entityName}')

In [0]:
if entityName == 'customers':
    # Define Dynamic Query:
    query = f"""
    SELECT CAST(SalesOrderNumber AS {columns[0]['dataType']}) AS {columns[0]['columnName']}
    ,      CAST(SalesOrderLineNumber AS {columns[1]['dataType']}) AS {columns[1]['columnName']})
    ,      CAST(OrderDate AS {columns[2]['dataType']}) AS {columns[2]['columnName']}
    ,      CAST(CustomerName AS {columns[3]['dataType']}) AS {columns[3]['columnName']}
    ,      CAST(EmailAddress AS {columns[4]['dataType']}) AS {columns[4]['columnName']}
    ,      CAST(Item AS {columns[5]['dataType']}) AS {columns[5]['columnName']}
    ,      CAST(Quantity AS {columns[6]['dataType']}) AS {columns[6]['columnName']}
    ,      CAST(UnitPrice AS {columns[7]['dataType']}) AS {columns[7]['columnName']}
    ,      CAST(TaxAmount AS {columns[8]['dataType']}) AS {columns[8]['columnName']}
    FROM tv_customers
    """

In [0]:
if entityName == 'products':
    # Define Dynamic Query:
    query = f"""
    SELECT CAST(ProductID AS {columns[0]['dataType']}) AS {columns[0]['columnName']}
    ,      CAST(ProductName AS {columns[1]['dataType']}) AS {columns[1]['columnName']})
    ,      CAST(Category AS {columns[2]['dataType']}) AS {columns[2]['columnName']}
    ,      CAST(ListPrice AS {columns[3]['dataType']}) AS {columns[3]['columnName']}
    FROM tv_products
    """

In [0]:
# Check data exists:
if df.count() > 0:
    # Set location for delta table:
    location = f"/{sourceSystemName}/{entityName}/{date_path}/"
    
    # Write to delta table:
    df.write.mode('overwrite').format('delta').option('header', 'true').save(SILVER + location)

    # Output location:
    dbutils.notebook.exit(location)
else:
    dbutils.notebook.exit(-1)